In [ ]:
""" Artificial Bee Colony Algorithm

Input
- D: number of decision variables
- S: swarm size
- T: number of cycles
- limit: decides when scouts phase needs to be executed, often taken Np*D

Np: number of food sources/employed bees/onlooker bees


Output


"""

function ArtificialBeeColonization(D,bounds_lower, bounds_upper,S, T, limit)
    Np = S/2
    
    # initialize population
    population = initialize_population(D, bounds_lower, bounds_upper, Np)
    
    # calculate objective values and fitness values for population
    objective_values = compute_objective(population, false)
    fitness_values = compute_fitness(objective_values)
    
    # initialize trial vector for population
    trial = zeros(Np, 1)
    
    
    for iterations in 1:length(T)
    
        ## EMPLOYED BEE PHASE
    
        pop_new_evolved, fitness_new_evolved,objective_new_evolved, trial_new = employed_bee_phase(population, bounds_lower, bounds_upper, trial)
        population = pop_new_evolved; fitness_values = fitness_new_evolved; trial = trial_new; objective_values = objective_new_evolved,
    
    
        ## ONLOOKER BEE PHASE
        probabilities = foodsource_info_prob(fitness_values)
        pop_new_evolved, fitness_new_evolved,objective_new_evolved,trial_new = onlooker_bee_phase(population, bounds_lower, bounds_upper, trial, probabilities)  
        population = pop_new_evolved; fitness_values = fitness_new_evolved; trial = trial_new; objective_values = objective_new_evolved,

        
        ## SCOUTING PHASE
        best_fitness_current_iteration = 
        best_fitness = maximum()
        Scouting(population,bounds_lower,bounds_upper,trial,fitness_values,objective_values,limit, best_fitness,D)
    
    end

    return optimal_solution
end	
